Import packages

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd
import csv
from numpy import genfromtxt
from math import pi
plt.ion()
dirname = '/home/msilvafe/Documents/SO/umux/Noise/'

Initialize Constants

In [4]:
k_b = 1.38e-23 #k Boltzmann [J/K]
phi_0 = 2.067833831e-15 #Flux Quantum in [Wb]
Ntones_MF = 1792.
Ntones_MF_db = 10*np.log10(Ntones_MF)
Ntones_HF = 1792.
Ntones_HF_db = 10*np.log10(Ntones_HF)
Ntones_LF = 1195.
Ntones_LF_db = 10*np.log10(Ntones_LF)
Ntones = 2000.
Ntones_db = 10*np.log10(Ntones)
N_MF = 12
N_HF = 6
N_LF = 3
Ntones_tot = N_MF*Ntones_MF+N_HF*Ntones_HF+N_LF*Ntones_LF
Ntones_tot_db = 10*np.log10(Ntones_tot)
Lambda = 0.333
dfdOhi0 = 2.17e5 # Hz/Phi_0
fmin = 4e9
fmax = 8e9
Q = 5e4
dfdI = 4.59e-2 # Hz/pA
LJ = 6.69e-11
#Inductance Matrix
Lsqsq = 2.23e-11
Lsqfr = 2.71e-11
Lsqin=2.28e-10
Lsqres=1.35e-12
Lfrfr=6.49e-10
Lfrin=3.96e-10
Lfrres=2.5e-12
Linin=5.48e-9
Linres=1.86e-11
Lresres=2.28e-10
Psmurfmax=0.
LNA_TN= [2.1,40.,191.]

Define Function for Reading In Coax Attenuation Data and Fit for slope and intercept

In [5]:
def readcoax(fname,f_eval):
    d = genfromtxt(fname, delimiter=',')
    slope_T1=(d[2,1]-d[1,1])/(d[2,0]-d[1,0])
    slope_T2=(d[2,2]-d[1,2])/(d[2,0]-d[1,0])
    int_T1=d[1,1]-slope_T1*d[1,0]
    int_T2=d[1,2]-slope_T2*d[1,0]
    dT1_feval=slope_T1*f_eval+int_T1
    dT2_feval=slope_T2*f_eval+int_T2
    slope = (dT1_feval-dT2_feval)/(d[0,1]-d[0,2])
    intercept = dT1_feval - slope*d[0,1]
    return [slope, intercept]

Read in config file for input coax chain

In [10]:
c = pd.read_csv(dirname+'InputChain.csv', delimiter=',')
d = np.transpose(c.values)
compin = d[0,:]
lossin = d[1,:]
Tavgin = (d[2,:]+d[3,:])/2
coax_YNin = d[4,:]
coax_typein = d[5,:]
coax_lin = d[6,:]
for i in range(len(compin)):
    if coax_YNin[i] == 'Y':
        t = readcoax(dirname+coax_typein[i]+'.csv',8)
        lossin[i] = (t[0]*Tavgin[i]+t[1])*coax_lin[i]

Read in config file for output coax chain

In [11]:
c = pd.read_csv(dirname+'OutputChain.csv',delimiter=',')
d = np.transpose(c.values)
compout = d[0,:]
lossout = d[1,:]
Tavgout = (d[2,:]+d[3,:])/2
coax_YNout = d[4,:]
coax_typeout = d[5,:]
coax_lout = d[6,:]
Gtot = 0.
for i in range(len(compout)):
    if coax_YNout[i] == 'Y':
        t = readcoax(dirname+coax_typeout[i]+'.csv',8)
        lossout[i] = (t[0]*Tavgout[i]+t[1])*coax_lout[i]

Define Function to refer noise temp through the chain

In [12]:
def ref_noise_temp(loss,temp,T0,p=False):
    T=T0
    loss_lin = 10.**(-loss/10.)
    for i in range(len(loss)):
        Tp = T*loss_lin[i] + temp[i]*(1-loss_lin[i])
        T=Tp
    if p==True:
        print(T0, 'K Noise Temp Referred to Feedline: ',T,' K')    
    return T

Calculate 300K referred

In [15]:
T_300K_Referred = ref_noise_temp(lossin,Tavgin,300.)
print('300K referred =', np.round(T_300K_Referred,2), 'K')

300K referred = 1.53 K


Define Function to Calculate the Power Referred to the Feedline

In [17]:
def p_referred_pertone(loss, Pin, p=False):
    Pout = Pin - sum(loss)
    if p==True:
        print('Power at Feedline: ',Pout,'dB')
    return Pout
#As an example we will print the power per tone at the feedline for 
#the maximum output of SMuRF 
Pfeed = p_referred_pertone(lossin,Psmurfmax-Ntones_db)
print('Maximum Power at Feedline Given SMuRF Limits = ',np.round(Pfeed,2),'dB')

Maximum Power at Feedline Given SMuRF Limits =  -65.22 dB


Define Function to Calculate Noise from DAC

In [36]:
def refer_phase_noise(dBc,loss,temp,Pin,k_b = k_b, Q = Q, f = fmax,p=False):
    T_out = (10.**((Pin-dBc)/10)*.001)/k_b
    T_feed = ref_noise_temp(loss,temp,T_out)
    P_feed = p_referred_pertone(loss,Pin)
    dBc_feed = 10*np.log10(k_b*T_feed/.001) - P_feed
    Sphi = 10**(dBc_feed/10)
    Sf = Sphi*(f/(2*Q))**2
    NEI = np.sqrt(Sf/(dfdI**2))
    if p == True:
        print('Phase Noise of: ',dBc, 'dBc/Hz referred to NEI:',NEI)
    return NEI

Define Function to Calculate the Noise Temperatures Referred to LNA-1 input (i.e. 300K noise referred to feed, LNA chain referred to LNA-1 input, pseudo noise referred to LNA-1 input, etc.)

In [19]:
def TN_to_NEI(T,LJ=LJ,f0=fmin,Min=Lsqin,phi_0=phi_0,alpha=1./2.,p=False):
    NEI = (1./np.sqrt(alpha))*np.sqrt((4.*k_b*T*LJ)/(pi*f0))*(1/Min)
    if p == True:
        print('Noise temp', T, 'K @ LNA-1 input referred to NEI:',np.round(NEI/1e-12,2),'pA/rt(Hz)')
    return NEI/1e-12

Define Function to Calculate the HEMT Pseudo Noise Floor from 3rd Order Nonlinearity

In [20]:
def Pseudo_Noise_Floor(Pin,loss,Dip,Ntones=Ntones,k_b=k_b):
    BWNL = 10*np.log10(12e9)
    NNLtones = 10*np.log10(Ntones*(Ntones-2))
    P_LNA1 = p_referred_pertone(loss,Pin)-Dip
    P_perspur_out = 3.*P_LNA1+100. #This comes from IP3 Measurements
    P_NL_out = P_perspur_out+NNLtones
    PNP_out = P_NL_out-BWNL
    PNP_in = PNP_out - 39 
    #This comes from the Gain of the 2 stage amplifiers shouldn't hard code this in the long run
    PNP_K = (10.**(PNP_in/10.)*.001)/k_b
    PNP_NEI = TN_to_NEI(PNP_K)
    return(PNP_NEI)

Define Function to Calculate the Referred HEMT Noise

In [25]:
def Total_Chain_Noise(loss_out):
    TNtot = 0.
    Gaintot_lin = 10**(np.sum((-loss_out))/10.)
    j=0
    loss_lin = 10.**(-loss_out/10.)
    for i in range(len(loss_out)):
        if loss_out[i]>0:
            TNtot = TNtot*loss_lin[i] + Tavgout[i]*(1.-loss_lin[i])
        if loss_out[i]<0:
            TNtot = (TNtot+LNA_TN[j])*loss_lin[i]
            j=j+1;
    TNout = TNtot/Gaintot_lin
    return TNout

In [26]:
Total_Chain_Noise(lossout)

3.3638684980029736

Define Function to add noise in quadrature

In [29]:
def quad_sum(a,b):
    out=np.sqrt(a**2+b**2)
    return out

Define Function to Sum All Noise Sources

In [41]:
def Calc_All_Noise(lossin = lossin, Tavgin = Tavgin, loss_out = lossout):
    N_PNP = Pseudo_Noise_Floor(-38,lossin,10)
    N_DAC = refer_phase_noise(101,lossin,Tavgin,-38)
    N_300K = TN_to_NEI(ref_noise_temp(lossin,Tavgin,300.))
    N_HEMT_TN = TN_to_NEI(Total_Chain_Noise(lossout))
    N_TES_bias = 10. #This number is just by design
    NEI_tot = quad_sum(quad_sum(quad_sum(N_PNP,N_DAC),quad_sum(N_300K,N_HEMT_TN)),N_TES_bias)
    return [N_PNP,N_DAC,N_300K,N_HEMT_TN,N_TES_bias],NEI_tot

In [42]:
Calc_All_Noise()

([0.5454726383664535,
  15.552910056806372,
  4.1623729592648155,
  6.167047897569229,
  10.0],
 19.93861529939447)

Now we have the framework to iterate through attenuators and look at the impact on total noise